In [76]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.notebook_job_step import NotebookJobStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker import session

# from sagemaker.s3 import S3Downloader
# from sagemaker.s3_utils import s3_path_join
# from sagemaker.utils import name_from_base
# from sagemaker.workflow import ParameterString
# from sagemaker.workflow.parameters import ParameterInteger, ParameterBoolean, ParameterString

sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
image_uri = "885854791233.dkr.ecr.us-east-1.amazonaws.com/sagemaker-distribution-prod:0-cpu"
kernel_name = "python3"
role = sagemaker.get_execution_role()

# subfolder_name = "notebook-step-artifacts-pipelines/"
# notebook_artifacts = f"s3://{default_bucket}/{subfolder_name}"
# notebook_artifacts

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [78]:
from sagemaker.workflow.retry import (
    StepRetryPolicy, 
    StepExceptionTypeEnum,
    SageMakerJobExceptionTypeEnum,
    SageMakerJobStepRetryPolicy
)

pipeline_name = "nb-job-steps-pipelines"
process_notebook_01 = "notebook_job_01.ipynb"
process_notebook_02 = "notebook_job_02.ipynb"
process_notebook_ensemble = "notebook_job_ensemble.ipynb"

# display_name = "MyNotebookSteps"
# process_job_name = "nb-preprocess"
# process_description = "This step downloads an NLP dataset and creates a CSV file out of it"
# process_step_name = "process-step"

# notebook job parameters
# nb_job_params = {"default_s3_bucket": notebook_artifacts}

process_nb_step1 = NotebookJobStep(
    input_notebook=process_notebook_01,
    image_uri=image_uri,
    kernel_name=kernel_name,
    retry_policies=[
        StepRetryPolicy(
            exception_types=[
                StepExceptionTypeEnum.SERVICE_FAULT, 
                StepExceptionTypeEnum.THROTTLING
            ],
            expire_after_mins=5,
            interval_seconds=10,
            backoff_rate=2.0 
        )
    ]
)

process_nb_step2 = NotebookJobStep(
    input_notebook=process_notebook_02,
    image_uri=image_uri,
    kernel_name=kernel_name,
    retry_policies=[
        StepRetryPolicy(
            exception_types=[
                StepExceptionTypeEnum.SERVICE_FAULT, 
                StepExceptionTypeEnum.THROTTLING
            ],
            expire_after_mins=5,
            interval_seconds=10,
            backoff_rate=2.0 
        )
    ],

    
    # instance_type="ml.m5.4xlarge",
    # name=preprocess_step_name,
    # description=preprocess_description,
    # notebook_job_name=preprocess_job_name,
    # image_uri=image_uri,
    # display_name=display_name,
    # role=role,
    # parameters=nb_job_params,
)

process_nb_step_ensemble = NotebookJobStep(
    input_notebook=process_notebook_ensemble,
    image_uri=image_uri,
    kernel_name=kernel_name,
    retry_policies=[
        StepRetryPolicy(
            exception_types=[
                StepExceptionTypeEnum.SERVICE_FAULT, 
                StepExceptionTypeEnum.THROTTLING
            ],
            expire_after_mins=5,
            interval_seconds=10,
            backoff_rate=2.0 
        )
    ],
    depends_on=[process_nb_step1,process_nb_step2],

    
    # instance_type="ml.m5.4xlarge",
    # name=preprocess_step_name,
    # description=preprocess_description,
    # notebook_job_name=preprocess_job_name,
    # image_uri=image_uri,
    # display_name=display_name,
    # role=role,
    # parameters=nb_job_params,
)




In [83]:
# create pipeline
pipeline = Pipeline(
    name=pipeline_name,
    steps=[process_nb_step_ensemble]
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [84]:
# pipeline.create(session.get_execution_role())
# pipeline.upsert(role_arn=session.get_execution_role())
pipeline.upsert(role)


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:541948677097:pipeline/nb-job-steps-pipelines',
 'ResponseMetadata': {'RequestId': '57d949bd-435a-42b2-a7cd-158cf1a31c99',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '57d949bd-435a-42b2-a7cd-158cf1a31c99',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Mon, 04 Mar 2024 07:43:27 GMT'},
  'RetryAttempts': 0}}

In [ ]:
execution = pipeline.start(parameters={})
execution.wait(delay=30, max_attempts=60)
execution_steps = execution.list_steps()
print(execution_steps)

In [ ]:
print(pipeline.describe()["PipelineArn"])
lastModifiedTime = pipeline.describe()["LastModifiedTime"]
formatted_dt = lastModifiedTime.strftime("%Y-%m-%d %H:%M")
print(formatted_dt)